In [1]:
import pandas as pd
from utilities.api_functions import *

# load the lookup table with real GVV IDs
geoid_lu_df = pd.read_csv('/Users/joshpaul/epa-justice/repo/epa-justice/tbl/NCRPlaces_Census_04192024.csv')
# optionally, limit the lookup table to a few locations to test all geography types
geoid_lu_df_test = geoid_lu_df[geoid_lu_df['name'].isin(['Eagle River', 'Fairbanks', 'Arctic Village', 'Fort Yukon'])]

In [2]:
geoid_lu_df_test

,id,name,alt_name,region,country,latitude,longitude,type,GEOIDFQ,PLACENAME,AREATYPE,COMMENT
10,AK124,Fairbanks,NaN,Alaska,US,64.8378,-147.716,community,0500000US02090,Fairbanks North Star Borough,County,Data represent information from nearest [COLUM...
45,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000201,Census Tract 2.01,Census tract,Data for this place represent multiple merged ...
46,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000202,Census Tract 2.02,Census tract,Data for this place represent multiple merged ...
47,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000204,Census Tract 2.04,Census tract,Data for this place represent multiple merged ...
48,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000205,Census Tract 2.05,Census tract,Data for this place represent multiple merged ...
49,AK103,Eagle River,NaN,Alaska,US,61.3221,-149.567,community,1400000US02020000206,Census Tract 2.06,Census tract,Data for this place represent multiple merged ...
77,AK22,Arctic Village,Vashrąįį K'ǫǫ,Alaska,US,68.1269,-145.538,community,1600000US0203990,Arctic Village CDP,Census designated place,Data represent information from nearest [COLUM...
166,AK130,Fort Yukon,Gwichyaa Zheh,Alaska,US,66.5647,-145.274,community,1600000US0226760,Fort Yukon city,Incorporated place,Data represent information from nearest [COLUM...


In [3]:
# use the individual fetch functions get data for GVV IDs in the test lookup table
# follow the printed URLs to QC individual values from the results table
for gvv_id in list(geoid_lu_df_test.id.unique()):
    dhc = fetch_census_data_and_compute("dhc", gvv_id, geoid_lu_df_test, print_url=True)
    print(dhc)
    acs5 = fetch_census_data_and_compute("acs5", gvv_id, geoid_lu_df_test, print_url=True)
    print(acs5)
    cdc = fetch_cdc_data_and_compute(gvv_id, geoid_lu_df_test, print_url=True)
    print(cdc)


Requesting US Census data from: https://api.census.gov/data/2020/dec/dhc?get=P12_001N,P12_002N,P12_026N,P12_003N,P12_004N,P12_005N,P12_006N,P12_020N,P12_021N,P12_022N,P12_023N,P12_024N,P12_025N,P12_027N,P12_028N,P12_029N,P12_030N,P12_044N,P12_045N,P12_046N,P12_047N,P12_048N,P12_049N&for=county:090&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
  GEOID  total_population  pct_65_plus  pct_under_18
0   090           95655.0        11.46         23.95
Requesting US Census data from: https://api.census.gov/data/2020/acs/acs5/subject?get=S1810_C03_001E,S1810_C03_001M,S2701_C03_001E,S2701_C03_001M,S2701_C05_001E,S2701_C05_001M&for=county:090&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
   pct_w_disability  moe_pct_w_disability  pct_insured  moe_pct_insured  \
0              11.1                   1.0         90.6              1.0   

   pct_uninsured  moe_pct_uninsured GEOID  
0            9.4                1.0   090  
Requesting CDC data from: https://data.cdc.gov/reso

In [5]:
# if test results look good, run the fetch and merge with the full lookup table and concatenate all results
# this will take a while (potentially 60 minutes for the entire dataset), as it makes many calls to the various APIs
# heavy API usage may be "throttled" by the service
results = []
for gvv_id in list(geoid_lu_df.id.unique()):
    results.append(fetch_and_merge(geoid_lu_df, gvv_id))
results_df = pd.concat(results)
results_df

In [ ]:
# save to CSV
results_df.to_csv('data_to_export.csv', index=False)